In [1]:
import pandas as pd

column_names = ['ano', 'insumos', 'agropecuária', 'industria', 'servicos', 'total']
sheet_cols = ['A:F', 'A,H:L', 'A,N:R', 'A,T:X', 'A,Z:AD', 'A,AF:AJ']
add_column = ['agronegocio', 'agricola', 'pecuario']

list_pib = []

n = 0
for cols in sheet_cols:
    df_pib = pd.read_excel(
        'raw/PIB_Cepea.xlsx',
        usecols=cols,
        skiprows=8,
        nrows=24,
        header=None,
        names=column_names
    )
    df_pib['ramo'] = add_column[n if n < 3 else n - 3]
    df_pib['pib_renda'] = True if n < 3 else False
    list_pib.append(df_pib)
    n += 1

df_pib = pd.concat(list_pib, ignore_index=True)

df_pib.head()

,ano,insumos,agropecuária,industria,servicos,total,ramo,pib_renda
0,1996,31597.663029,200211.777672,514755.882623,646536.982537,1.393102e+06,agronegocio,True
1,1997,32825.998197,199911.301734,482894.842674,604714.559307,1.320347e+06,agronegocio,True
2,1998,31818.410647,200969.741431,455966.002147,576123.413175,1.264878e+06,agronegocio,True
3,1999,34784.067160,206952.587184,453199.228768,573119.731296,1.268056e+06,agronegocio,True
4,2000,40682.954859,209175.390473,496957.470676,626206.339831,1.373022e+06,agronegocio,True


In [2]:
df_prod = pd.read_excel(
    'raw/produção_2019.xlsx',
    skiprows=6,
    nrows=26,
    names=['uf', 'area_abril', 'area_maio', 'area_junho', 'prod_abril', 'prod_maio', 'prod_junho']
)

df_prod.head()

,uf,area_abril,area_maio,area_junho,prod_abril,prod_maio,prod_junho
0,ALAGOAS,45 842,80 283,80 385,95 917,105 458,105 525
1,AMAPÁ,21 548,22 442,23 242,50 813,60 375,62 955
2,AMAZONAS,22 930,22 930,22 930,41 207,41 420,41 420
3,BAHIA,3 130 065,3 107 865,3 085 865,8 283 660,9 065 031,9 359 331
4,CEARÁ,909 518,961 926,961 640,559 791,918 483,903 649


In [3]:
temp = df_prod[
    ['area_abril', 'area_maio', 'area_junho', 'prod_abril', 'prod_maio', 'prod_junho']
].applymap(lambda _: _.replace(' ', '')).astype('int')
temp['uf'] = df_prod['uf']
df_prod = temp
df_prod.head()

,area_abril,area_maio,area_junho,prod_abril,prod_maio,prod_junho,uf
0,45842,80283,80385,95917,105458,105525,ALAGOAS
1,21548,22442,23242,50813,60375,62955,AMAPÁ
2,22930,22930,22930,41207,41420,41420,AMAZONAS
3,3130065,3107865,3085865,8283660,9065031,9359331,BAHIA
4,909518,961926,961640,559791,918483,903649,CEARÁ


In [4]:
from sqlalchemy import create_engine
from decouple import config

USER_DB = config("USER_DB", default='postgres')
PASSWORD_DB = config("PASSWORD_DB", default='password')
URL_DB = config("URL_DB", default='localhost')
NAME_DB = config("NAME_DB", default='db_name')
PORT_DB = config("PORT_DB", default='5432')
DB_URL = f"postgresql://{USER_DB}:{PASSWORD_DB}@{URL_DB}:{PORT_DB}/{NAME_DB}"


engine = create_engine(DB_URL)

df_pib.to_sql(name='PIP_Cepea', con=engine, if_exists='replace')
df_prod.to_sql(name='producao_2019', con=engine, if_exists='replace')